## Connect to SSH

In [24]:
import sys

import paramiko
from getpass import getpass
from os.path import exists
import os

# Check to see if output file exists. If file exists, ask user if they want to delete it.
if exists('results.body') is True:
    choice = input('Results File already exists! Would you like to delete it? (Y/N): ')
    if choice == 'Y':
        os.remove('results.body')
    elif choice == 'N':
        print('Continuing with existing results file present. WARNING: New resutls will be appended onto old results file.')
    else:
        print('Not a valid option. Goodbye')
        sys.exit()

# Create pass prompt
thePass = getpass(prompt="Enter your password: ")

# Host Info
host = "192.168.6.71"
port = "2222"
username = "samuel.johnson01"
password = thePass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication Failed")

## Upload and Run Kraken

In [69]:
# Start SFTP Session
sftp = ssh.open_sftp()

# List of files to upload
upfiles = ['kraken']

for eachFile in upfiles:
    local_path = '/home/samuel.johnson01/' + eachFile
    sftp.put(eachFile, local_path)

# Run the uploaded script
command = ['sudo -S chmod +x kraken', 'sudo -S ./kraken --folder /usr/bin  --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin']

for eachCMD in command:
    # Get the output from the command
    stdin, stout, stderr = ssh.exec_command(eachCMD)

    stdin.write(thePass + "\n")
    # Get results from output
    lines = stout.readlines()

    # Convert the list to a string.
    output = ''.join(lines)

    cmd_output = output

    print(cmd_output)


sftp.close()


INFO[0000] Looking for configuration file with name config.yaml 
INFO[0000] No configuration file found, generating a default one... 
INFO[0000] Loading Yara rules...                        
INFO[0000] Scanning running processes...                
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=3138 process=ls
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=25680 process=snapd
INFO[0014] Scanning autoruns...                         
WARN[0015] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
WARN[0015] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
INFO[0016] Scanning the filesystem (this can take several minutes)... 
ERRO[0016] Some malicious artifacts have been detected on this system: 
ERRO[0016] Found detection for foundGoBinary            
ERRO[0016] Found detection for foundGoBinary            
ERRO[0016] Found detection for foundGoBinary     

## Run LSOF Against suspicious PIDs

In [77]:
# List to enter suspicious PIDs
pid = ['3138']

for val in pid:
    cmd = "sudo -S lsof -p "+val
    stdin, stout, stderr = ssh.exec_command(cmd)

    stdin.write(thePass + "\n")
    # Get results from output
    lines = stout.readlines()
    for line in lines:
        print(line)
    # Convert the list to a string.
    output = ''.join(lines)
    cmd_output = output

COMMAND  PID     USER   FD      TYPE DEVICE SIZE/OFF   NODE NAME

ls      3138 p.dalton  cwd       DIR    8,1     4096 258130 /home/p.dalton/test-lk

ls      3138 p.dalton  rtd       DIR    8,1     4096      2 /

ls      3138 p.dalton  txt       REG    8,1  6597697 258133 /usr/local/bin/ls

ls      3138 p.dalton  DEL       REG    8,1            3453 /usr/lib/x86_64-linux-gnu/libc-2.31.so

ls      3138 p.dalton  DEL       REG    8,1            3466 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so

ls      3138 p.dalton  DEL       REG    8,1            3449 /usr/lib/x86_64-linux-gnu/ld-2.31.so

ls      3138 p.dalton    0r      CHR    1,3      0t0      6 /dev/null

ls      3138 p.dalton    1w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log

ls      3138 p.dalton    2w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log

ls      3138 p.dalton    3r      CHR    1,3      0t0      6 /dev/null

ls      3138 p.dalton    4u      REG    8,1        4 258134 /home/p.dalton/test-